## Speed comparison: torch_kmeans vs. scikit-learn


### TL;DR

**Use torch_kmeans on GPU.**
It is the fastest in all relevant cases with speed-up of close to two magnitudes (~70x) for some cases.

### Summary

fastest < yx mid < yx slowest

where yx is the approximate multiple of execution time w.r.t. the fastest model
1. BS=1, N=20, D=2: torch_kmeans(CPU) < 2x torch_kmeans(GPU) < 3x scikit-learn
2. BS=1, N=500, D=2: torch_kmeans(GPU) < 2x scikit-learn < 12x torch_kmeans(CPU)
3. BS=1, N=100, D=2: torch_kmeans(GPU) < 2x scikit-learn < 4x torch_kmeans(CPU)
4. BS=10, N=100, D=2: torch_kmeans(GPU) < 15x scikit-learn < 28x torch_kmeans(CPU)
5. BS=64, N=100, D=2: torch_kmeans(GPU) < 60x scikit-learn < 70x torch_kmeans(CPU)
6. BS=1, N=100, D=20: torch_kmeans(GPU) < 2x torch_kmeans(CPU) < 2.5x scikit-learn
7. BS=1, N=100, D=200: torch_kmeans(GPU) < 15x scikit-learn < 20x torch_kmeans(CPU)

Run locally on a Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz and a NVIDIA GeForce 1080ti.


In [32]:
!pip install torch-kmeans


In [1]:
# imports
import numpy as np
import torch
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans as SKLearnKMeans
from torch_kmeans import KMeans as TorchKMeans


In [2]:
# function to generate some clustering data
def get_data(bs: int = 1,
             n: int = 20,
             d: int = 2,
             k: int = 4,
             different_k: bool = False,
             k_lims = (2, 5),
             add_noise: bool = True,
             fp_dtype = torch.float32,
             seed: int = 42):
    torch.manual_seed(seed)
    if different_k:
        a, b = k_lims
        k = torch.randint(low=a, high=b, size=(bs,)).long()
    else:
        k = torch.empty(bs).fill_(k).long()

    # generate pseudo clustering data
    x, y = [], []
    for i, k_ in enumerate(k.numpy()):
        x_, y_ = make_blobs(n_samples=n, centers=k_, n_features=d, random_state=seed+i)
        x.append(x_)
        y.append(y_)
    x = torch.from_numpy(np.stack(x, axis=0))
    y = torch.from_numpy(np.stack(y, axis=0))
    if add_noise:
        x += torch.randn(x.size())

    return x.to(fp_dtype), y, k




In [3]:
# wrapper for scikit-learn to process full batch provided
def eval_sklearn_km(m, x):
    return [m.fit(x_).labels_ for x_ in x]


In [4]:
def eval_torch_km(m, x):
    return m(x)


### 1) 1 instance(s), 20 points, 2 dimensions, 8 inits, K=2

In [5]:
N_INIT = 8
N_ITERS = 200
TOL = 1e-5
SEED = 123
BS, N, D = (1, 20, 2)
K = 2
assert torch.cuda.is_available()
device = torch.device("cuda")

x_pt, _, _ = get_data(bs=BS, n=N, d=D, k=K)
x_cuda = x_pt.clone().to(device=device)
x_np = x_pt.clone().numpy()

m_pt = TorchKMeans(
    max_iter=N_ITERS,
    n_clusters=K,
    init_method='rnd',
    num_init=N_INIT,
    tol=TOL,
    seed=SEED,
    verbose=False,
)

m_skl = SKLearnKMeans(
        max_iter=N_ITERS,
        n_clusters=K,
        init='random',
        n_init=N_INIT,
        tol=TOL,
        verbose=False,
        random_state=SEED,
    )


def eval_pt():
    eval_torch_km(m_pt, x_pt)


def eval_cuda():
    eval_torch_km(m_pt, x_cuda)


def eval_skl():
    eval_sklearn_km(m_skl, x_np)


In [6]:
%timeit -r 5 -n 200 eval_skl()

4.12 ms ± 70.7 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [7]:
%timeit -r 5 -n 200 eval_pt()

1.52 ms ± 336 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [8]:
%timeit -r 5 -n 200 eval_cuda()

2.38 ms ± 1.04 ms per loop (mean ± std. dev. of 5 runs, 200 loops each)


### 2) 1 instance(s), 500 points, 2 dimensions, 8 inits, K=20

In [9]:
N_INIT = 8
N_ITERS = 200
TOL = 1e-4
SEED = 123
BS, N, D = (1, 500, 2)
K = 20
assert torch.cuda.is_available()
device = torch.device("cuda")

x_pt, _, _ = get_data(bs=BS, n=N, d=D, k=K)
x_cuda = x_pt.clone().to(device=device)
x_np = x_pt.clone().numpy()

m_pt = TorchKMeans(
    max_iter=N_ITERS,
    n_clusters=K,
    init_method='rnd',
    num_init=N_INIT,
    tol=TOL,
    seed=SEED,
    verbose=False,
)

m_skl = SKLearnKMeans(
        max_iter=N_ITERS,
        n_clusters=K,
        init='random',
        n_init=N_INIT,
        tol=TOL,
        verbose=False,
        random_state=SEED,
    )


def eval_pt():
    eval_torch_km(m_pt, x_pt)


def eval_cuda():
    eval_torch_km(m_pt, x_cuda)


def eval_skl():
    eval_sklearn_km(m_skl, x_np)


In [10]:
%timeit -r 5 -n 200 eval_skl()

22.2 ms ± 556 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [11]:
%timeit -r 5 -n 200 eval_pt()

152 ms ± 893 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [12]:
%timeit -r 5 -n 200 eval_cuda()

12.6 ms ± 728 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


### 3) 1 instance(s), 100 points, 2 dimensions, 8 inits, K=8

In [13]:
N_INIT = 8
N_ITERS = 200
TOL = 1e-4
SEED = 123
BS, N, D = (1, 100, 2)
K = 8
assert torch.cuda.is_available()
device = torch.device("cuda")

x_pt, _, _ = get_data(bs=BS, n=N, d=D, k=K)
x_cuda = x_pt.clone().to(device=device)
x_np = x_pt.clone().numpy()

m_pt = TorchKMeans(
    max_iter=N_ITERS,
    n_clusters=K,
    init_method='rnd',
    num_init=N_INIT,
    tol=TOL,
    seed=SEED,
    verbose=False,
)

m_skl = SKLearnKMeans(
        max_iter=N_ITERS,
        n_clusters=K,
        init='random',
        n_init=N_INIT,
        tol=TOL,
        verbose=False,
        random_state=SEED,
    )


def eval_pt():
    eval_torch_km(m_pt, x_pt)


def eval_cuda():
    eval_torch_km(m_pt, x_cuda)


def eval_skl():
    eval_sklearn_km(m_skl, x_np)


In [14]:
%timeit -r 5 -n 200 eval_skl()

7.08 ms ± 452 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [15]:
%timeit -r 5 -n 200 eval_pt()

11 ms ± 338 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [16]:
%timeit -r 5 -n 200 eval_cuda()

3.28 ms ± 294 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


### 4) 10 instances, 100 points, 2 dimensions, 8 inits, K=8

In [17]:
N_INIT = 8
N_ITERS = 200
TOL = 1e-4
SEED = 123
BS, N, D = (10, 100, 2)
K = 8
assert torch.cuda.is_available()
device = torch.device("cuda")

x_pt, _, _ = get_data(bs=BS, n=N, d=D, k=K)
x_cuda = x_pt.clone().to(device=device)
x_np = x_pt.clone().numpy()

m_pt = TorchKMeans(
    max_iter=N_ITERS,
    n_clusters=K,
    init_method='rnd',
    num_init=N_INIT,
    tol=TOL,
    seed=SEED,
    verbose=False,
)

m_skl = SKLearnKMeans(
        max_iter=N_ITERS,
        n_clusters=K,
        init='random',
        n_init=N_INIT,
        tol=TOL,
        verbose=False,
        random_state=SEED,
    )


def eval_pt():
    eval_torch_km(m_pt, x_pt)


def eval_cuda():
    eval_torch_km(m_pt, x_cuda)


def eval_skl():
    eval_sklearn_km(m_skl, x_np)


In [18]:
%timeit -r 5 -n 200 eval_skl()

89.4 ms ± 338 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [17]:
%timeit -r 5 -n 200 eval_pt()

154 ms ± 3.18 ms per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [19]:
%timeit -r 5 -n 200 eval_cuda()



6.54 ms ± 741 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


### 5) 64 instances, 100 points, 2 dimensions, 8 inits, K=8

In [20]:
N_INIT = 8
N_ITERS = 200
TOL = 1e-4
SEED = 123
BS, N, D = (64, 100, 2)
K = 8
assert torch.cuda.is_available()
device = torch.device("cuda")

x_pt, _, _ = get_data(bs=BS, n=N, d=D, k=K)
x_cuda = x_pt.clone().to(device=device)
x_np = x_pt.clone().numpy()

m_pt = TorchKMeans(
    max_iter=N_ITERS,
    n_clusters=K,
    init_method='rnd',
    num_init=N_INIT,
    tol=TOL,
    seed=SEED,
    verbose=False,
)

m_skl = SKLearnKMeans(
        max_iter=N_ITERS,
        n_clusters=K,
        init='random',
        n_init=N_INIT,
        tol=TOL,
        verbose=False,
        random_state=SEED,
    )


def eval_pt():
    eval_torch_km(m_pt, x_pt)


def eval_cuda():
    eval_torch_km(m_pt, x_cuda)


def eval_skl():
    eval_sklearn_km(m_skl, x_np)


In [21]:
%timeit -r 5 -n 200 eval_skl()

599 ms ± 11.1 ms per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [22]:
%timeit -r 5 -n 200 eval_pt()

701 ms ± 11.2 ms per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [23]:
%timeit -r 5 -n 200 eval_cuda()

9.72 ms ± 389 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


### 6) 1 instances, 100 points, 20 dimensions, 8 inits, K=8

In [24]:
N_INIT = 8
N_ITERS = 200
TOL = 1e-4
SEED = 123
BS, N, D = (1, 100, 20)
K = 8
assert torch.cuda.is_available()
device = torch.device("cuda")

x_pt, _, _ = get_data(bs=BS, n=N, d=D, k=K)
x_cuda = x_pt.clone().to(device=device)
x_np = x_pt.clone().numpy()

m_pt = TorchKMeans(
    max_iter=N_ITERS,
    n_clusters=K,
    init_method='rnd',
    num_init=N_INIT,
    tol=TOL,
    seed=SEED,
    verbose=False,
)

m_skl = SKLearnKMeans(
        max_iter=N_ITERS,
        n_clusters=K,
        init='random',
        n_init=N_INIT,
        tol=TOL,
        verbose=False,
        random_state=SEED,
    )


def eval_pt():
    eval_torch_km(m_pt, x_pt)


def eval_cuda():
    eval_torch_km(m_pt, x_cuda)


def eval_skl():
    eval_sklearn_km(m_skl, x_np)


In [25]:
%timeit -r 5 -n 200 eval_skl()

5.29 ms ± 307 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [26]:
%timeit -r 5 -n 200 eval_pt()

4.28 ms ± 408 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [27]:
%timeit -r 5 -n 200 eval_cuda()

2.35 ms ± 275 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


### 7) 1 instances, 100 points, 200 dimensions, 8 inits, K=8

In [28]:
N_INIT = 8
N_ITERS = 200
TOL = 1e-4
SEED = 123
BS, N, D = (10, 100, 2)
K = 8
assert torch.cuda.is_available()
device = torch.device("cuda")

x_pt, _, _ = get_data(bs=BS, n=N, d=D, k=K)
x_cuda = x_pt.clone().to(device=device)
x_np = x_pt.clone().numpy()

m_pt = TorchKMeans(
    max_iter=N_ITERS,
    n_clusters=K,
    init_method='rnd',
    num_init=N_INIT,
    tol=TOL,
    seed=SEED,
    verbose=False,
)

m_skl = SKLearnKMeans(
        max_iter=N_ITERS,
        n_clusters=K,
        init='random',
        n_init=N_INIT,
        tol=TOL,
        verbose=False,
        random_state=SEED,
    )


def eval_pt():
    eval_torch_km(m_pt, x_pt)


def eval_cuda():
    eval_torch_km(m_pt, x_cuda)


def eval_skl():
    eval_sklearn_km(m_skl, x_np)


In [29]:
%timeit -r 5 -n 200 eval_skl()

89.9 ms ± 526 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [30]:
%timeit -r 5 -n 200 eval_pt()

115 ms ± 1.55 ms per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [31]:
%timeit -r 5 -n 200 eval_cuda()

6.34 ms ± 427 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)
